<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/12_finetuning_llama3_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMによるテキストの埋め込み


## autoregressive LM for text embedding
* パラメータ数が7~8BのLLMをテキスト分類に使うとき・・・
* 普通はクラスラベルをテキストとして出力させる。
  * autoregressive modelの普通の使い方は、やはりテキストの生成。
* しかし、あえてLLMをテキストの埋め込みに使ってみる。
  * つまり、BERTと同じような使い方をする。
* また、埋め込みのためのモデルとしてのファインチューニングも行う。

## インストール

In [ ]:
!pip install -U transformers datasets bitsandbytes accelerate peft trl

## インポート

In [ ]:
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import (
    set_seed,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from transformers.models.llama.modeling_llama import (
    LlamaForSequenceClassification,
)
from transformers.modeling_outputs import ModelOutput
from peft import LoraConfig
from trl import SFTTrainer

set_seed(123)

In [ ]:
dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
    shuffle=True,
    trust_remote_code=True,
)
num_categories = len(set(dataset["train"]["category"]))

max_seq_length = 512

* 今回はtitleを使う。

In [ ]:
dataset["train"]["title"][:10]

In [ ]:
category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

## 分類モデルの定義
* テキストの末尾のトークンに対応する出力を分類に使う。
* 今回使うLLMのクラスを継承して、新たなクラスを定義する。
* 今回使うLLMは`LlamaForSequenceClassification`
  * https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py

In [ ]:
class LivedoorNet(LlamaForSequenceClassification):
  def __init__(self, *args, **kwargs):
    super(LivedoorNet, self).__init__(*args, **kwargs)

  def forward(
      self,
      input_ids,
      category=None,
      attention_mask=None,
      output_attentions=None,
      output_hidden_states=None,
      return_dict=None,
      inputs_embeds=None,
      labels=None,
  ):
    outputs = super(LivedoorNet, self).forward(
        input_ids,
        attention_mask=attention_mask,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )
    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(outputs.logits, category)
    return ModelOutput(
        loss=loss,
        logits=outputs.logits,
        past_key_values=outputs.past_key_values,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )

## モデルの取得
* 今回は`tokyotech-llm/Llama-3-Swallow-8B-Instruct-v0.1`を使う。
  * https://huggingface.co/tokyotech-llm/Llama-3-Swallow-8B-Instruct-v0.1
* NF4量子化とDouble Quantizationの詳細は下の論文を参照。
  * https://arxiv.org/abs/2305.14314
* 量子化については下の記事も参考になる。
  * https://huggingface.co/blog/4bit-transformers-bitsandbytes

In [ ]:
model_name = "tokyotech-llm/Llama-3-Swallow-8B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = LivedoorNet.from_pretrained(
    model_name,
    num_labels=num_categories,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_cache=False, # https://github.com/huggingface/transformers/issues/33489
)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)

# pad_tokenをeos_tokenに設定しないと、
# 各トークン列の末尾のトークンではなく、
# ミニバッチの中の最も長いトークン列の末尾で、
# 分類用のlogitを取得してしまう。
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
model

## LoRAの設定

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

## trainerの設定

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    output_dir="outputs_cls",
    max_steps=1000,
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    learning_rate=5e-5,
    eval_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

## trainerの作成

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="title",
    peft_config=peft_config,
    max_seq_length=max_seq_length,
)

* 以下を実行しないと、エラーが出る。
  * trainerを作成するとき、カテゴリの情報が消されてしまうため。

In [ ]:
trainer.train_dataset = trainer.train_dataset.add_column("category", dataset["train"]["category"])
trainer.eval_dataset = trainer.eval_dataset.add_column("category", dataset["validation"]["category"])

## trainableなパラメータの確認

In [ ]:
def print_trainable_parameters(model, verbose=False):
  trainable_params = 0
  all_param = 0
  for name, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
      if verbose: print(name)
  print(
      f"trainable params: {trainable_params} "
      f"|| all params: {all_param} "
      f"|| trainable%: {100 * trainable_params / all_param}"
  )

In [ ]:
print_trainable_parameters(trainer.model)

In [ ]:
print_trainable_parameters(trainer.model, verbose=True)

## 正解率を計算するヘルパ関数

In [ ]:
def evaluate_by_accuracy(model, tokenizer, dataset, batch_size=4):
  model.eval()
  num_correct_answers = 0
  num_answers = 0
  for i in tqdm(range(0, len(dataset), batch_size)):
    examples = dataset[i:i+batch_size]
    encoding = tokenizer(
        examples["title"],
        padding=True,
        return_tensors="pt",
        )
    category = torch.tensor(examples["category"])
    with torch.no_grad():
      outputs = model.forward(**encoding, category=category)
    num_correct_answers += (outputs.logits.argmax(-1) == category).sum()
    num_answers += len(examples["category"])
  model.train()
  return num_correct_answers / num_answers

* ファインチューニングする前に評価してみる。
  * scoreレイヤが未学習なので性能はランダム分類に近い。
    * RTX4090なら次のセルは30秒で終わる。

In [ ]:
evaluate_by_accuracy(model, tokenizer, dataset["validation"])

## LLMのファインチューニング

In [ ]:
trainer.train()

## 評価

In [ ]:
evaluate_by_accuracy(model, tokenizer, dataset["validation"])